In [ ]:
import os

import torch
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

from preprocess import get_csv

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# Set the file path for your CSV file
file_path = 'arxiv-metadata-oai-snapshot.csv'

# Check if the file exists
if not os.path.exists(file_path):
    # If the file does not exist, call the get_csv method
    print("Creating CSV from JSON file.")
    get_csv()
else:
    print("The file already exists.")

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(df.head())

In [ ]:
corpus = df["abstract"]
title = df["title"]

# Set the file path for your CSV file
embeddings_path = 'embeddings.npy'

# Check if the file exists
if not os.path.exists(embeddings_path):
    # If the file does not exist, call the get_embedding method
    print("Creating CSV from JSON file.")
    corpus_embeddings = model.encode(corpus, show_progress_bar=True)
    np.save("./embeddings.npy", corpus_embeddings, allow_pickle=True)
else:
    print("The file already exists.")
    corpus_embeddings = np.load("./embeddings.npy", allow_pickle=True)

print(corpus_embeddings.shape)

In [ ]:
### We need to make this cell into a function.

num_clusters = 30
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

# clustered_sentences = [[] for i in range(num_clusters)]
# for sentence_id, cluster_id in enumerate(cluster_assignment):
#     clustered_sentences[cluster_id].append(title[sentence_id])

# # for i, cluster in enumerate(clustered_sentences):
# #     print("Cluster ", i + 1)
# #     print(cluster)
# #     print("")

In [ ]:
print(cluster_assignment)
np.save("./clusters.npy", cluster_assignment, allow_pickle=True)

In [ ]:
df["clustering-{}".format(num_clusters)] = cluster_assignment
df

In [7]:
df.to_csv("arxiv-metadata-oai-snapshot-clustering-{}.csv".format(num_clusters), index=False)

In [ ]:
df = pd.read_csv("arxiv-metadata-oai-snapshot-clustering-{}.csv".format(num_clusters))
df

In [ ]:
df[df["clustering-30"] == 0]

In [38]:
x = df['versions'].copy()
for i in range(len(x)):
    x[i] = int(x[i][:4])
    
np.save("./years.npy", x, allow_pickle=True)